In [1]:
# Importing the data and basic packages
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# df = pd.read_csv('creditcard.csv')
# df = df.drop('Time', axis=1)
# df.reset_index(inplace=True)

In [2]:
# df.info()

In [3]:
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# plt.style.use('fivethirtyeight')
# plt.rcParams['figure.figsize'] = (12, 6)

# df = pd.read_csv('creditcard.csv')
# df = df.drop('Time', axis=1)
# df.reset_index(inplace=True)

# target_name = 'Class'
# X = df.drop(target_name, axis =1)
# sc_X = StandardScaler()
# X = sc_X.fit_transform(X)
# y = pd.DataFrame(df[target_name])

# # Split into train and test sets
# # Is it recommended to stratify in order to keep the same proportion of 1s and 0s
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# dtree = tree.DecisionTreeClassifier()
#     #class_weight='balanced')
#                                     #min_weight_fraction_leaf=0.01)
# dtree = dtree.fit(X_train,y_train)

# # Plot!

# importances = dtree.feature_importances_
# feat_names = df.drop(['Class'], axis=1).columns

# indices = np.argsort(importances)[::-1]
# plt.figure(figsize=(12,6))
# plt.title("Which features should we keep?")
# plt.bar(range(len(indices)), importances[indices], color='lightblue', align='center')
# plt.step(range(len(indices)), np.cumsum(importances[indices]), where='mid', label='Cumulative')
# plt.xticks(range(len(indices)), feat_names[indices], rotation='vertical', fontsize=14)
# plt.xlim([-1, len(indices)])
# plt.show()

does not mean sklearn pipeline in the strictiest sense

actually a CLASS that contains an sklearn pipeline

combine Undersampler and Oversampler in one function using an if statement

# Pipeline Steps

### add option to input either a whole df or X and y
#### maybe also the option to input whole DF and then specify the target variable
* train_test_split(stratified) + conversion of sets to DMatrices
* transform data to dictionary
* Imputer
### how about missing values for categorical variables?????
* StandardScaler
* DictVectorizer
* AVOIDING DUMMY VARIABLE TRAP
* Function - Resampling (Undersampling or SMOTE)
* MAYBE Feature Selection
* Train Initial XGBoost Model
* Function: Hyperparameter Tuning using just the Training Set
    * GridSearchCV / RandomizedSearchCV
    * Training a new Model using best hyperparameters (return: tuned_model)
* Function: XGBoost Cross-validation

### Shorted tuning code
* by using nested ifs to determine the function name, parameter name, and parameter value

### target variable
* target is None, use the last column as y
* if it is provided, use that as y and drop it to form X

methods for the class:
.train()
.create()
.validate()
.train_create_validate()
.plot()
#### .classvariable_override()
* method for forcibly setting the new new cls.parameter given as the new self.parameter
#### .tune()
* class decorator as a method for changing the default class parameters

#### .preprocessor__
* attribute for extracting just the preprocessing pipeline

#### model evaluation has its own preprocessing pipeline fitted
* add an optional parameter in fit_transform for channeling an untouched trained model through the preprocessing pipeline until it hits the cross-validation step

## .linear_learn()
* for training linear base model with CV

## early stopping!!!!!!


# BUILD A FUNCTION FOR VISUALIZING DECISION BOUNDARIES

In [4]:
# import packages
import xgboost as xgb
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import Imputer, LabelEncoder, OneHotEncoder
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, hamming_loss,\
jaccard_similarity_score, log_loss, matthews_corrcoef, precision_score, recall_score, zero_one_loss,\
explained_variance_score, mean_absolute_error, mean_squared_error, mean_squared_log_error, median_absolute_error, r2_score

from sklearn.feature_selection import SelectFromModel
from imblearn.under_sampling import NearMiss
from imblearn.pipeline import make_pipeline
import operator

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler, minmax_scale, RobustScaler, \
                                    MaxAbsScaler, QuantileTransformer
from sklearn import tree

C:\Users\JanErish\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
# #### define a function for undersampling
# def Binary_Undersampler(df):
#     """This class is applicable only on data for binary classification.
#     This function assumes that 1 is the value of the underrepresented class."""
#     Positive_count = df.iloc[:,-1].sum()
#     Positive_indices = np.array(df[df.iloc[:,-1] ==1].index)
#     Negative_indices = df[df.iloc[:,-1] == 0].index
#     random_Negative_indices = np.array(np.random.choice(Negative_indices, Positive_count, replace = False))
#     Undersample_indices = np.concatenate([Positive_indices, random_Negative_indices])
#     Undersampled_df = df.loc[Undersample_indices,:]
#     return Undersampled_df

In [6]:
# XGBoost Model Trainer (custom decorator function)

def XGBoost_Trainer(df, target=None, model= 'other', resampling=None,
                    booster="gbtree", objective="reg:linear",
                    test_size=0.2,
                    random_state=69,
                    normalization=True,
                    pca=False,
                    hyperparameter_test= "none",
                    cv_metric="rmse",
                    missing_values="NaN",
                    imputation_strategy="mean",
                    n_iter= 25,
                    hypertune_folds= 5,
                    cv_folds= 5,
                    scoring="neg_mean_squared_error",
                    grid_params = "preset",
                    random_params = "preset"):
    """This function requires the xgboost package to be imported as xgb."""
    
    if model == "Tree Regressor":
        bj = ['gbtree','reg:linear']
    elif model == "Linear Regressor":
        bj = ['gblinear','reg:linear']
    elif model == "Tree Classifier":
        bj = ['gbtree', 'reg:logistic']
    elif model == "Linear Classifier":
        bj = ['gblinear', 'reg:logistic']
    elif mode == 'other':
        bj = [booster, objective]
    
    grid_parameters = {}
    random_parameters = {}
    
    if grid_params == "preset":
        grid_params = grid_parameters
    if random_params == "preset":
        random_params == random_parameters
    
    regression_objectives=[]
    classification_objectives = ["reg:logistic"]

    # Data Preprocessing Pipeline Steps
    preprocessing_steps = [
            ("Imputer", Imputer(missing_values=missing_values, strategy= imputation_strategy, axis=0))
                            ]
    
    # Scaling
    if normalization == True:
        preprocessing_steps = preprocessing_steps + [
            ("Scaler", StandardScaler())
                            ]
    
    # Dummy Variable Creation
    class Dictionator:
        def __init__(self, orient="records"):
            self.orient = orient
        
        def fit(self, X, y):
            return self
    
        def transform(self, X, y):
            return X.to_dict(self.orient), y.to_dict(self.orient)
        
        def fit_transform(self, X, y):
            self.fit(X, y)
            return self.transform(X, y)
    
    
    preprocessing_steps = preprocessing_steps + [ 
        ("Encoder", FeatureUnion(
            [("Label",LabelEncoder()),
             ("Hot", OneHotEncoder())]))
                            ]  
    
    #### define a CLASS for undersampling
    class Binary_Undersampler:
        """This class is applicable only on data for binary classification.
        This class assumes that 1 is the value of the underrepresented class."""
        def fit(self, X, y):
            return self
    
        def transform(self, X, y):   
            self.Positive_count = y.values.sum()
            self.Positive_indices = np.array(y[y.iloc[:,-1] ==1].index)
            self.Negative_indices = y[y.iloc[:,-1] == 0].index
            self.random_Negative_indices = np.array(np.random.choice(self.Negative_indices, self.Positive_count,
                                                                     replace = False))
            self.Undersample_indices = np.concatenate([self.Positive_indices, self.random_Negative_indices])
            return X.loc[self.Undersample_indices,:], y.loc[self.Undersample_indices,:]
        
        def fit_transform(self, X, y):
            self.fit(X, y)
            return self.transform(X, y)

    
    # Resampling Method
    if objective in classification_objectives or model in ["Tree Classifier","Linear Classifier"]:
        if resampling == "Undersampling":
            preprocessing_steps = preprocessing_steps + [("The Undersampler", Binary_Undersampler())]
        elif resampling == "SMOTE":
            preprocessing_steps = preprocessing_steps + [("The Oversampler", SMOTE())]
            
    # Principal Component Analysis (PCA)
    if pca == True:
        preprocessing_steps = preprocessing_steps + [("PCA", PCA())]
    
    # Separating features and target
    X, y = df[df.columns.tolist()[:,-1]], df[df.columns.tolist()[-1]]
    
    # Split between test and training sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)

    
    # Completing the pipeline with the appropriat XGBoost Model
    # correct version
    if bj[0] == "gblinear":
        DM_train = xgb.DMAtrix(data=X_train, label=y_train)
        if hyperparameter_test == "Grid Search":
            pipeline = Pipeline[
                preprocessing_steps,
                ("Tuner", GridSearchCV(estimator=xgb.train(dtrain=DM_train, booster=bj[0], objective=bj[1]),
                                      param_grid= grid_params,
                                      scoring= scoring,
                                      cv= hypertune_folds,
                                      verbose=1))
                        ]
        elif hyperparameter_test == "Random Search":
            pipeline = Pipeline[
                preprocessing_steps,
                ("Tuner", RandomizedSearchCV(estimator=xgb.train(dtrain=DM_train, booster=bj[0], objective=bj[1]),
                                                                param_distributions=random_parameters,
                                                                scoring= scoring,
                                                                cv = hypertune_folds,
                                                                n_iter= n_iter))
                        ]
        elif hyperparameter_test == "none":
            pipeline = Pipeline[
                preprocessing_steps,
                ("Model", xgb.train(dtrain=DM_train, booster=bj[0], objective=bj[1]))
                        ]
    
    elif bj[0] == "gbtree":
        if bj[1] in regression_objectives:
            if hyperparameter_test == "Grid Search":
                pipeline = Pipeline[
                    preprocessing_steps,
                    ("Tuner", GridSearchCV(estimator=xgb.XGBRegressor(booster=bj[0], objective=bj[1]),
                                          param_grid= grid_params,
                                          scoring= scoring,
                                          cv= hypertune_folds,
                                          verbose=1))
                        ]
            elif hyperparameter_test == "Random Search":
                pipeline = Pipeline[
                    preprocessing_steps,
                    ("Tuner", RandomizedSearchCV(estimator=xgb.XGBRegressor(booster=bj[0], objective=bj[1]),
                                                                           param_distributions=random_parameters,
                                                                           scoring= scoring,
                                                                           cv = hypertune_folds,
                                                                           n_iter= n_iter))
                        ]
            elif hyperparameter_test == "none":
                pipeline = Pipeline[
                    preprocessing_steps,
                    ("Model", xgb.XGBRegressor(booster=bj[0], objective=bj[1]))
                        ]

        elif bj[1] in classification_objectives:
            if hyperparameter_test == "Grid Search":
                pipeline = Pipeline[
                    preprocessing_steps,
                    ("Tuner", GridSearchCV(estimator=xgb.XGBClassifier(booster=bj[0], objective=bj[1]),
                                          param_grid= grid_params,
                                          scoring= scoring,
                                          cv= hypertune_folds,
                                          verbose=1))
                        ]
            elif hyperparameter_test == "Random Search":
                pipeline = Pipeline[
                    preprocessing_steps,
                    ("Tuner", RandomizedSearchCV(estimator=xgb.XGBClassifier(booster=bj[0], objective=bj[1]),
                                                                           param_distributions=random_parameters,
                                                                           scoring= scoring,
                                                                           cv = hypertune_folds,
                                                                           n_iter= n_iter))
                        ]
            elif hyperparameter_test == "none":
                pipeline = Pipeline[
                    preprocessing_steps,
                    ("Model", xgb.XGBClassifier(booster=bj[0], objective=bj[1]))
                        ]             
    
    
    
    # fitting the pipeline 
    pipeline.fit(X_train.to_dict("records"), y_train)
    
    if hyperparameter_test == "Grid Search" or hyperparameter_test == "Random Search":
        if bj[0] == "gblinear":
            xgb_model = xgb.XGBRegressor(booster="gbtree", objective="reg:linear", **pipeline.best_params_)
            xgb_model.fit(X_train, y_train)
        elif model == "Linear Regressor":
            xgb_model == xgb.train(booster="gblinear", objective='reg:linear', **pipeline.best_params_)
            xgb_model.fit(X_train, y_train)
        elif model == "Tree Classifier":
            xgb_model == XGBClassifier(booster="gbtree")
    


# EDIT SO THAT COLUMN NAMES WILL BE PRESERVED EVEN AFTER IMPUTATION_NATION

In [7]:
# Imputer Class

class Imputation_Nation:
        """This class handles the imputation of missing values
        and the encoding and creation of dummy variables for categorical variables.

        If the target variable is passed as y, it will also be imputed if impute_y= True.
        Otherwise, y will pass through as untouched."""
        
        classification_objectives = ['reg:logistic','binary:logistic', 'binary:logitraw',
                            'multi:softmax', 'multi:softprob', 'rank:pairwise' ]
        
        missing_values = None
        num_imput_strat = None
        cat_imput_strat = None
        imputation_axis = None
        objective = None
        impute_y = None
        Cat_mask = None
        Num_cols = None
        
        def __init__(self, impute_y=False, objective="reg:linear",missing_values='NaN', num_imput_strat='mean', 
                     cat_imput_strat='most_frequent', imputation_axis=0,
                    Cat_mask = None, Num_cols = None,
                    
                     classification_objectives=None):
        
            # assigning parameters as instance variables
            varses = list(vars(Imputation_Nation).keys())
            self.variables = varses[len(varses) - varses[::-1].index('__doc__') : varses.index('__init__')]
            class_name = "Imputation_Nation"+"."
            for v in self.variables:
                # if the class variable for the argument is not empty, assign its value as the instance variable
                if eval("%s != None" % (class_name+v)) is True:
                    exec("self.%s = %s" % (v,class_name+v))
                # if the class variable is empty, assign to instance the value passed as argument during instantiation
                else:
                    exec("self.%s = %s" % (v, v)) 

        def fit(self, X, y=None):
            return self
        
        def transform(self, X, y=None):
                
            X_full = pd.DataFrame(X)
            if y is not None:
                y_full = pd.DataFrame(y)
                
            # identify categorical columns
            if self.Cat_mask is None:
                Cat_mask = X_full.dtypes == object
                Cat_cols = X_full.columns[Cat_mask].tolist()
            else:
                Cat_mask = self.Cat_mask
                Cat_cols = X_full.columns[Cat_mask].tolist()
            
            # identify numeric columns
            if self.Num_cols is None:
                Num_cols = X_full.select_dtypes(exclude=['object']).columns.tolist()
            else:
                Num_cols = self.Num_cols
            
            # Encoding and imputing categorical variables
            if len(Cat_cols) != 0:
                for col in X_full.loc[:,Cat_mask].columns:
                    L_E = LabelEncoder()
                    IMP = Imputer(missing_values= 'NaN', strategy='most_frequent', axis=0)
                    le = L_E.fit(X_full.loc[:,col])
                    X_full.loc[:,col] = le.transform(X_full.loc[:,col])
                    IMP = IMP.fit(X_full.loc[:,col].values.reshape(-1, 1))
                    X_full.loc[:,col] = IMP.transform(X_full.loc[:,col].values.reshape(-1, 1)).astype(int)
                    X_full.loc[:,col] = le.inverse_transform(X_full.loc[:,col])
                    
                # Creating dummy variables for categorical data
                #X_full = OneHotEncoder(categorical_features=Cat_mask, sparse=False).fit_transform(X_full)
                X_full = pd.get_dummies(X_full, drop_first=True, columns=Cat_cols)
            
            # Imputing numeric columns
            if len(Num_cols) != 0:
                X_full[Num_cols] = Imputer(missing_values=self.missing_values,strategy=self.num_imput_strat, 
                                axis=self.imputation_axis).fit_transform(X_full[Num_cols])
            
            if y is not None and self.impute_y == True:
                if self.objective in self.classification_objectives:
                    
                    # Encoding target variables formatted as string
                    if y_full.iloc[:,0].dtype == object:
                        Target_is_string = True
                    else:
                        Target_is_string = False
                    
                    if Target_is_string == True:
                        L_E = LabelEncoder()
                        L_E = L_E.fit(y_full.iloc[:,0].astype(str))
                        y_full.iloc[:,0] = L_E.transform(y_full.iloc[:,0].astype(str))
                    
                    IMP = Imputer(missing_values= 'NaN', strategy='most_frequent', axis=0)
                    MP = IMP.fit(y_full.iloc[:,0].values.reshape(-1, 1))
                    y_full.iloc[:,0] = IMP.transform(y_full.iloc[:,0].values.reshape(-1, 1)).astype(int)
                    
                    # If target was originally string, return to original string values
                    if Target_is_string == True:
                        y_full.iloc[:,0] = L_E.inverse_transform(y_full.iloc[:,0])
                    
                    # Creating dummy variables
                    # y_full = pd.get_dummies(y_full, drop_first=True, columns=[y_full.columns[0]])
                    # y_full = y_full.astype(int)
                    # Create a dictionary map for encoded labels for dependent variable
                    # self.target_mapping = dict(zip(L_E.classes_, L_E.transform(L_E.classes_)))
                
                else:
                    y_full.iloc[:,0] = Imputer(missing_values=self.missing_values,strategy=self.num_imput_strat, 
                                axis=self.imputation_axis).fit_transform(y_full.iloc[:,0])
                    
            if y is not None:
                return pd.DataFrame(X_full), pd.DataFrame(y_full)
            return pd.DataFrame(X_full)
        
        def fit_transform(self, X, y=None):
            self.fit(X, y)
            return self.transform(X, y)

In [8]:
class Nullifier:

    def find(self, a, valid_values):
        a = pd.DataFrame(a) 
        valid_indeces = a.iloc[:,0].isin(valid_values)
        if len(valid_indeces) != 0:
            invalid_a = pd.DataFrame(a.drop(a.index[valid_indeces]))
            self.null_values = list(invalid_a.iloc[:,0].unique())
    
    def finder(self, a, valid_values):
        self.find(a, valid_values)
        return self.null_values
    
    def nullify(self, a):
        a = pd.DataFrame(a)
        if len(self.null_values) != 0:
            for null_val in self.null_values:
                a = a.replace([null_val], np.nan)
        return a
    
    def find_nullify(self, a, valid_values):
        self.find(a, valid_values)
        return self.nullify(a)
    
    # DataFrame nullifier
    def df_nullifier(self, df, valid_dictionary):
        """ This method should be given a pandas DataFrame and a dictionary comprising of the following key/value pairs:
            Key =  column name
            Value = list of valid values for the column."""
        for col, valid_vals in valid_dictionary.items():
            df[col] = self.find_nullify(df[col], valid_vals)
        return df
    
    def df_null_dictionary(self, df, valid_dictionary):
        """Returns a dictionary of values considered null for each DatFrame column"""
        self.null_dictionary = {}
        for col, valid_vals in valid_dictionary.items():
            self.null_dictionary[col] = self.finder(df[col], valid_vals)
        return self.null_dictionary

In [9]:
class MCAR_Dropper:
    """This transformer is not compatible with sklearn Pipeline.
    
    This class automatically drops rows with missing data for a categorical target/dependent variable
    under the assumption that the data is Missing Completely at Random (MCAR).
    
    Parameter Null_values can be fed a list of label values that will be considered Null and thus will be dropped"""
    def __init__(self, null_values=None):
        self.null_values = null_values
    def fit(self, X, y):
        return self
    def transform(self, X, y):
        X_nonull = X
        y_nonull = y
        null_inds = pd.isnull(y).any(1).nonzero()[0]
        if len(null_inds) != 0:
            X_nonull = pd.DataFrame(X.drop(X.index[null_inds])).reset_index(drop=True)
            y_nonull = pd.DataFrame(y.drop(y.index[null_inds])).reset_index(drop=True)
        
        # drop values considered as null based on given list
        if self.null_values is not None:
            for value in self.null_values:
                null_inds = y_nonull.iloc[:,0] == value
                if len(null_inds) != 0:
                    X_nonull = pd.DataFrame(X_nonull.drop(X_nonull.index[null_inds])).reset_index(drop=True)
                    y_nonull = pd.DataFrame(y_nonull.drop(y_nonull.index[null_inds])).reset_index(drop=True)
        
        return X_nonull, y_nonull
    def fit_transform(self, X, y):
        self.fit(X, y)
        return self.transform(X, y)

In [10]:
# ORIGINAL FEATURE SCALER CLASS
class Feature_Scaler:
    """Options for scaling methods (parameter "scale=") are the following:
        Normal, Standard, MinMax, Normal, Robust, MaxAbs, and Quantile.
        
            If the fit_transform method is given one positional argument (X) and target=None, 
            the class assumes that it is a DataFrame with the last
            column as the target variable and the rest as the features."""
    
    def __init__(self, scale="Standard", copy=True, with_mean=True, with_std=True,
                feature_range=(0, 1),
                 norm='l2',
                 with_centering=True, with_scaling=True, quantile_range=(25.0, 75.0),
                 n_quantiles=1000, output_distribution='uniform', ignore_implicit_zeros=False,
                 subsample=100000, random_state=69
                ):
        self.scale = scale
        
        if self.scale == "Standard":
            self.copy = copy
            self.with_mean = with_mean
            self.with_std = with_std
            self.scaler = StandardScaler(copy = self.copy, with_mean = self.with_mean, with_std = self.with_std)
        elif self.scale == "MinMax":
            self.feature_range= feature_range
            self.copy = copy
            self.scaler = MinMaxScaler(feature_range = self.feature_range, copy = self.copy)
        elif self.scale == "Normal":
            self.norm = norm
            self.copy = copy
            self.scaler = Normalizer(norm = self.norm, copy = self.copy)
        elif self.scale == "Robust":
            self.with_centering = with_centering
            self.with_scaling = with_scaling
            self.quantile_range = quantile_range
            self.copy = copy
            self.scaler = RobustScaler(with_centering = self.with_centering, with_scaling = self.with_scaling,
                                     quantile_range=self.quantile_range, copy=self.copy)
        elif self.scale == "MaxAbs":
            self.copy = copy
            self.scaler == MaxAbsScaler(copy=self.copy)
        elif self.scale == "Quantile":
            self.n_quantiles = n_quantiles
            self.output_distribution = output_distribution
            self.ignore_implicit = ignore_implicit
            self.subsample = subsample
            self.random_state = random_state
            self.copy = copy
            self.scaler == QuantileTransformer(n_quantiles=self.n_quantiles, output_distribution=self.output_distribution,
                                             ignore_implicit=self.ignore_implicit, subsample=self.subsample,
                                             random_state=self.random_state)
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
  
        X_scaled = pd.DataFrame(self.scaler.fit_transform(X), columns=X.columns)
        
        if y is not None:
            return X_scaled, pd.DataFrame(y)
        return X_scaled
        
    
    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X, y)

# HYPERSCALER TUNING @@@@@@@@@@@@

# Feature_Hyperscaler:

In [11]:
# Feature Scaler

class Feature_Hyperscaler:
    """This transformer is not compatible with the current sklearn Pipeline.
    
        Options for scaling methods (parameter "scale=") are the following:
        Normal, Standard, MinMax, Normal, Robust, MaxAbs, and Quantile.
        
        If scale = 'Hyperscale',
            every scaling method will be applied and tested
            to determine which method best contributes to model performance.
            
            best_params = {dictionary of best parameters for model tuning continuity}
            
            scorer = pass desired Sklearn.metrics scorer as a string
                passing scorer='Auto' entails the use of built-in XGBoost evaluation metrics
            
            The scaling method that contributes to the best score will be applied to the data to be returned.
        
        This feature selection transformer works only with any of the three boosters as base learners:
        -gbtree
        -gblinear
        -dart
        
        This class requires that X (features or independent variables) has already been encoded and Imputed. 
        
        Tuning requires the target (y) to also be passed.
        If target = 'insert target name' is passed, X = dataframe disregarding position of target column.
        If the fit_transform method is given one positional argument (X) and target=None, 
            the class assumes that it is a DataFrame with the last
            column as the target variable and the rest as the features."""
    
    # Compatible model evaluation metrics: Sklearn metrics
    scoring_functions = {'accuracy_score':accuracy_score,'f1_score':f1_score, 'hamming_loss':hamming_loss,
        'jaccard_similarity_score':jaccard_similarity_score, 'log_loss':log_loss, 'matthews_corrcoef':matthews_corrcoef,
                 'precision_score':precision_score, 'recall_score':recall_score, 'zero_one_loss':zero_one_loss,
                'explained_variance_score':explained_variance_score, 'mean_absolute_error':mean_absolute_error,
                 'mean_squared_error':mean_squared_error, 'mean_squared_log_error':mean_squared_log_error,
                 'median_absolute_error':median_absolute_error, 'r2_score':r2_score}

    classification_objectives = ['reg:logistic','binary:logistic', 'binary:logitraw',
                                'multi:softmax', 'multi:softprob', 'rank:pairwise' ]
    regression_objectives = ['reg:linear','count:poisson','reg:gamma', 'reg:tweedie']
    
    # metrics with which higher value = higher model performance
    the_higher_the_better = ['accuracy_score','f1_score','jaccard_similarity_score',
                                  'precision_score','recall_score',
                                'explained_variance_score','r2_score','Average test auc',
                                'Average test ndcg','Average test map']
    # metrics with which lower value = higher model performance
    the_lower_the_better = ['hamming_loss', 'log_loss','matthews_corrcoef','zero_one_loss','mean_absolute_error',
                           'mean_squared_error','mean_squared_log_error','median_absolute_error',
                           'Average test error','Average test rmse','Average test mae','Average test log loss',
                           'Average test merror','Average test mlogloss']

    # parameters to inherited from mother class
    scale=None
    best_params=None
    booster=None
    objective=None
    scorer=None
    copy=None
    with_mean=None
    with_std=None
    feature_range=None
    norm=None
    with_centering=None
    with_scaling=None
    quantile_range=None
    n_quantiles=None
    output_distribution=None
    ignore_implicit_zeros=None
    subsample=None
    random_state=None
    test_size=None
    
    def __init__(self, scale="Hyperscale", best_params={},
                 booster='gbtree', objective='reg:logistic', scorer='accuracy_score',
                 copy=True, with_mean=True, with_std=True,
                feature_range=(0, 1),
                 norm='l2',
                 with_centering=True, with_scaling=True, quantile_range=(25.0, 75.0),
                 n_quantiles=1000, output_distribution='uniform', ignore_implicit_zeros=False,
                 subsample=100000, random_state=69, test_size=0.2,
                # values passed for these parameters will never be used unless: class.parameter is made None
                scoring_functions=None, classification_objectives=None,
                 the_higher_the_better=None, the_lower_the_better=None):
        
        # assigning parameters as instance variables
        varses = list(vars(Feature_Hyperscaler).keys())
        self.variables = varses[len(varses) - varses[::-1].index('__doc__') : varses.index('__init__')]
        class_name = "Feature_Hyperscaler"+"."
        for v in self.variables:
            # if the class variable for the argument is not empty, assign its value as the instance variable
            if eval("%s != None" % (class_name+v)) is True:
                exec("self.%s = %s" % (v,class_name+v))
            # if the class variable is empty, assign to instance the value passed as argument during instantiation
            else:
                exec("self.%s = %s" % (v, v))
        
        # determining the model build used for feature selection
        if self.booster == 'gbtree':
            if self.objective in Feature_Hyperscaler.classification_objectives:
                self.best_params['objective'] = self.objective
                if 'booster' in list(self.best_params.keys()):
                    del self.best_params['booster']
                self.model = xgb.XGBClassifier(**self.best_params)
            elif self.objective in Feature_Hyperscaler.regression_objectives:
                self.best_params['objective'] = self.objective
                if 'booster' in list(self.best_params.keys()):
                    del self.best_params['booster']
                self.model = xgb.XGBRegressor(**self.best_params)
        elif self.booster == 'dart' or self.booster == 'gblinear':
            self.best_params['booster'] = self.booster
            self.best_params['objective'] = self.objective
        
        self.scaler_params={'StandardScaler':[StandardScaler,{'copy':self.copy, 
                                              'with_mean':self.with_mean, 'with_std':self.with_std}],
                          'MinMaxScaler':[MinMaxScaler,{'feature_range':self.feature_range, 'copy':self.copy}],
                          'Normalizer':[Normalizer,{'norm':self.norm, 'copy':self.copy}],
                            'RobustScaler':[RobustScaler,{'with_centering':self.with_centering,
                                            'with_scaling':self.with_scaling,
                                             'quantile_range':self.quantile_range, 'copy':self.copy}],
                           'MaxAbsScaler':[MaxAbsScaler,{'copy':self.copy}],
                           'QuantileTransformer':[QuantileTransformer,{'n_quantiles':self.n_quantiles, 
                                                  'output_distribution':self.output_distribution,
                                                  'ignore_implicit_zeros':self.ignore_implicit_zeros,
                                                  'subsample':self.subsample,'random_state':self.random_state}]}
        
        # Assigning the scaler if scale is not "Hyperscale"
        if self.scale is not 'Hyperscale':
            self.scaler = self.scaler_params[self.scale][0](**self.scaler_params[self.scale][1])
    
    def fit(self, X, y=None, target=None):
        if y is None:
            if self.target is None:
                y = pd.DataFrame(X.iloc[:,-1])
                X = pd.DataFrame(X.drop(X.columns[[-1,]], axis=1))
            else:
                y = pd.DataFrame(X[self.target])
                X = X.drop(target, axis =1)
        
        if self.scale == 'Hyperscale':
            # Encode labels if string when needed
            if self.booster == 'dart' and self.objective in Feature_Selector.classification_objectives\
                                                                    or self.scorer == 'f1_score':
                if y.iloc[:,0].dtype == object:
                    Target_is_string = True
                else:
                    Target_is_string = False

                if Target_is_string == True:
                    L_E = LabelEncoder()
                    L_E = L_E.fit(y.iloc[:,0].astype(str))
                    y.iloc[:,0] = L_E.transform(y.iloc[:,0].astype(str))

            # fit model on all training data            
            X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                test_size=self.test_size, random_state=self.random_state, stratify=y)
            
            scales = ['None','StandardScaler','MinMaxScaler','Normalizer',
                      'RobustScaler','MaxAbsScaler','QuantileTransformer']
            Scores = []
            for scale in scales:
                if scale is 'None':
                    X_train_scaled = X_train
                    X_test_scaled = X_test
                else:
                    scaler = self.scaler_params[scale][0](**self.scaler_params[scale][1])
                    X_train_scaled = scaler.fit_transform(X_train)
                    X_test_scaled = scaler.fit_transform(X_test)
                
                if self.booster is 'gbtree':
                    if self.objective in Feature_Hyperscaler.classification_objectives:
                        estimator = xgb.XGBClassifier
                    if self.objective in Feature_Hyperscaler.regression_objectives:
                        estimator = xgb.Regressor
                    selection_model = estimator(**self.best_params)
                    selection_model.fit(X_train_scaled, y_train.values.ravel(),
                                       eval_set=[(X_train_scaled,y_train.values.ravel()),
                                                 (X_test_scaled,y_test.values.ravel())], verbose=False)
                    # eval model
                    # using build-in evaluation metrics automatically matched with objective
                    if self.scorer == 'Auto':
                        result = selection_model.evals_result()
                        scorer_used = list(result['validation_1'].keys())[0]
                        score = np.mean(result['validation_1'][scorer_used])
                        Scores.append(score)

                    # using Sklearn metrics
                    else:
                        scoring_function = Feature_Hyperscaler.scoring_functions[self.scorer]
                        y_pred = selection_model.predict(X_test_scaled)
                        if y_pred.dtype != object:
                            predictions = [round(value) for value in y_pred]
                            y_test_used = [round(value) for value in y_test.iloc[:,0]]
                        else:
                            predictions = y_pred
                            y_test_used = y_test
                        score = scoring_function(y_test_used, predictions)
                        if self.scorer == 'accuracy_score':
                            score = score*100.00
                        Scores.append(score)
                        
                elif self.booster == 'dart' or self.booster == 'gblinear':
                    dtrain_scaled = xgb.DMatrix(data=X_train_scaled, label=y_train)
                    dtest_scaled = xgb.DMatrix(data=X_test_scaled, label=y_test)

                    # eval model
                    # using build-in evaluation metrics automatically matched with objective
                    if self.scorer == 'Auto':
                        result = {}
                        selection_model = xgb.train(dtrain=dtrain_scaled, params=self.best_params,
                                                    evals=[(dtest_scaled, 'eval')], evals_result=result,
                                                   verbose_eval=False)
                        scorer_used = list(result['eval'].keys())[0]
                        score = np.mean(result['eval'][scorer_used])
                        Scores.append(score)

                    # using Sklearn metrics
                    else:
                        scoring_function = Feature_Hyperscaler.scoring_functions[self.scorer]

                        selection_model = xgb.train(dtrain=dtrain_scaled, params=self.best_params)
                        y_pred = selection_model.predict(dtest_scaled)
                        if y_pred.dtype != object:
                            predictions = [round(value) for value in y_pred]
                            y_test_used = [round(value) for value in y_test.iloc[:,0]]
                        else:
                            predictions = y_pred
                            y_test_used = y_test
                        score = scoring_function(y_test_used, predictions)
                        if self.scorer == 'accuracy_score':
                            score = score*100.00
                        Scores.append(score)

            # building table of performance scores
            if self.scorer == 'Auto':
                self.scorer = 'Average test ' + scorer_used
            self.performance_scores = pd.DataFrame()
            self.performance_scores['Scalers'] = scales
            self.performance_scores[self.scorer] = Scores

            # Best scaler: returns max possible model performance 
            if self.scorer in Feature_Selector.the_higher_the_better:
                self.best_perf = self.performance_scores[self.performance_scores[self.scorer]\
                                                         == max(self.performance_scores[self.scorer])]
            elif self.scorer in Feature_Selector.the_lower_the_better:
                self.best_perf = self.performance_scores[self.performance_scores[self.scorer]\
                                                         == min(self.performance_scores[self.scorer])]
                
            # assigning the best-performing scaler as scaler
            self.best_scaler = self.best_perf.iloc[0,0]
            if self.best_scaler is not 'None':
                self.scaler = self.scaler_params[self.best_scaler][0](**self.scaler_params[self.best_scaler][1])
                # Fitting the scaler
                self.scaler.fit(X)
            else:
                self.scaler = None
            
            # reverse label encoding
            if self.booster == 'dart' and self.objective in Feature_Selector.classification_objectives:
                if Target_is_string == True:
                    y.iloc[:,0] = L_E.inverse_transform(y.iloc[:,0])
            
            return self
            
        else:
            # Fitting the scaler
            self.scaler.fit(X)
            return self
    
    def transform(self, X, y=None, target=None):
        if y is None:
            if target is None:
                y = pd.DataFrame(X.iloc[:,-1])
                X = pd.DataFrame(X.drop(X.columns[[-1,]], axis=1))
            else:
                y = pd.DataFrame(X[target])
                X = X.drop(target, axis =1)
        if self.best_scaler is not 'None':
            X_scaled = pd.DataFrame(self.scaler.transform(X), columns=X.columns)
            return X_scaled, pd.DataFrame(y)
        else:
            return pd.DataFrame(X), pd.DataFrame(y)
        
    def fit_transform(self, X, y=None, target=None):
        self.fit(X,y, target=target)
        return self.transform(X, y, target=target)

In [12]:
sclr = Feature_Hyperscaler(scale='StandardScaler')
sclr.scaler

StandardScaler(copy=True, with_mean=True, with_std=True)

In [13]:
# # Create the DMatrix: housing_dmatrix
# dmatrix = xgb.DMatrix(data=X, label=y)

# # Create the parameter dictionary: params
# params = {"booster": "gblinear", objective":"reg:linear","max_depth":4}

# # Train the model: xg_reg
# xg_reg = xgb.train(dtrain=dmatrix, params=params, num_boost_round=10)

# # Plot the feature importances
# xgb.plot_importance(xg_reg)

# ################
# xgb_params = {"objective": "reg:linear", "eta": 0.01, "max_depth": 8, "seed": 42, "silent": 1}
# num_rounds = 1000

#dtrain = xgb.DMatrix(X_full, label=y_full)
#gbdt = xgb.train(xgb_params, dtrain, num_rounds)

#importance = gbdt.get_fscore()#fmap='xgb.fmap')
#importance = sorted(importance.items(), key=operator.itemgetter(1), reverse=True)
#importance = pd.DataFrame(importance)

#xgb.plot_importance(gbdt)
#plt.show()

# self.test_model = self.model
#             self.test_model.fit(X_train, y_train.values.ravel())

#             # make predictions for test data and evaluate
#             y_pred = self.test_model.predict(X_test)
#             if y.iloc[:,0].dtype != object:
#                 predictions = [round(value) for value in y_pred]
#             else:
#                 predictions = y_pred
#             accuracy = accuracy_score(y_test, predictions)

#             # Fit model using each importance as a threshold
#             thresholds = sorted(self.test_model.feature_importances_)

In [14]:
#         for v in range(0, len(self.variables)-1):
#             if self.class_vars[v] is not None:
#                 self.instance_variables[v] = self.class_vars[v]
#             else:
#                 self.instance_variables[v] = self.arguments[v]

#         variables = ['booster', 'best_params', 'objective', 'random_state', 'test_size', 'scorer']
#         for var in variables:
#             instance_variable = eval("self."+var)
#             class_variable = eval('Feature_Selector.'+var)
#             variable = eval(var)
#             if class_variable is not None:
#                 instance_variable = class_variable
#             else:
#                 instance_variable = variable
        
#         self.booster=booster
#         self.best_params=best_params
#         self.objective=objective
#         self.random_state=random_state
#         self.test_size=test_size
#         self.scorer=scorer

"""HOW ON EARTH DOES ONE GET NAMES OF ARGUMENTS PASSED THROUGH __INIT___"""
class dat:
    apple = 'a'
    bottom = 'b'
    def __init__(self,a=1, b=2, c=3):
        self.a = a
        self.b = b
        self.c = c

        self.class_variables = list(vars(dat).keys())
    def varses(self,run=1):
        return self.class_variables

# dat().varses(2)
"""THIS ONE ONLY WORKS ON FUNCTIONS"""
#dat.__code__.co_varnames

AP = 1
A = "A"
P = "P"
word = eval(A + P)
print(word)

#arguments
a = 'argument'
b = 'argument'

#instance
ia = None
ib = None

#class
ca = 'class'
cb = None

variables = ['a', 'b']
for var in variables:
    instance_variable = eval("i"+var)
    print(instance_variable)
    class_variable = eval('c'+var)
    print(class_variable)
    variable = eval(var)
    print(variable)
    if class_variable is not None:
        instance_variable = class_variable
    else:
        instance_variable = variable
print(ia)
print(ib)
da = dat()
print(callable(getattr(dat(), 'varses')))
# two ways to get lists of class variables
members = [attr for attr in dir(da) if not callable(getattr(da, attr)) and not attr.startswith("__")]
members
print(list(vars(dat).keys()))
variables = list(dat().varses())
# getting values from a list between two specified values
variables = variables[len(variables) - variables[::-1].index('__module__') : variables.index('__init__')]
variables

# getting boolean values using eval()
eval(A+"=="+P)

1
None
class
argument
None
None
argument
None
None
True
['__module__', 'apple', 'bottom', '__init__', 'varses', '__dict__', '__weakref__', '__doc__']


False

In [15]:
AB = 1
variables =['B']
class_name = "A"
for v in variables:
    if eval("%s == None" % (class_name+v)) is True:
        print('Success')
    else:
        print('Fail')

Fail


In [16]:
# # Feature Selection Class # #

class Feature_Selector:
    """This transformer is not compatible with the current sklearn Pipeline.
    
    This feature selection transformer works only with the following boosters as base learners:
        -gbtree
        -dart
                                                                            
    Sklearn.metrics scoring functions can be passed to parameter:
        scorer = 'scoring_function'
    
    Built-in XGBoost evaluation metrics will be used by passing:
        scorer = 'Auto'                                                    
    
    This class requires that X (features or independent variables) has already been encoded and Imputed.
    
    Tuning requires the target (y) to also be passed.
        If target = 'insert target name' is passed, X = dataframe disregarding position of target column.
        If the fit_transform method is given one positional argument (X) and target=None, 
            the class assumes that it is a DataFrame with the last
            column as the target variable and the rest as the features."""
    
    # Compatible model evaluation metrics: Sklearn metrics
    scoring_functions = {'accuracy_score':accuracy_score,'f1_score':f1_score, 'hamming_loss':hamming_loss,
        'jaccard_similarity_score':jaccard_similarity_score, 'log_loss':log_loss, 'matthews_corrcoef':matthews_corrcoef,
                 'precision_score':precision_score, 'recall_score':recall_score, 'zero_one_loss':zero_one_loss,
                'explained_variance_score':explained_variance_score, 'mean_absolute_error':mean_absolute_error,
                 'mean_squared_error':mean_squared_error, 'mean_squared_log_error':mean_squared_log_error,
                 'median_absolute_error':median_absolute_error, 'r2_score':r2_score}

    classification_objectives = ['reg:logistic','binary:logistic', 'binary:logitraw',
                                'multi:softmax', 'multi:softprob', 'rank:pairwise' ]
    regression_objectives = ['reg:linear','count:poisson','reg:gamma', 'reg:tweedie']
    
    # metrics with which higher value = higher model performance
    the_higher_the_better = ['accuracy_score','f1_score','jaccard_similarity_score',
                                  'precision_score','recall_score',
                                'explained_variance_score','r2_score','Average test auc',
                                'Average test ndcg','Average test map']
    # metrics with which lower value = higher model performance
    the_lower_the_better = ['hamming_loss', 'log_loss','matthews_corrcoef','zero_one_loss','mean_absolute_error',
                           'mean_squared_error','mean_squared_log_error','median_absolute_error',
                           'Average test error','Average test rmse','Average test mae','Average test log loss',
                           'Average test merror','Average test mlogloss']
    
    booster=None
    best_params=None
    objective=None
    random_state=None
    test_size=None
    scorer=None

    def __init__(self, booster='gbtree', objective='reg:logistic', 
                 random_state=69, test_size=0.2, best_params={}, scorer='accuracy_score',
                
                 # values passed for these parameters will never be used unless: class.parameter is made None
                 scoring_functions=None, classification_objectives=None, regression_objectives=None,
                the_higher_the_better=None, the_lower_the_better=None):
        
        
        # assigning parameters as instance variables
        varses = list(vars(Feature_Selector).keys())
        self.variables = varses[len(varses) - varses[::-1].index('__doc__') : varses.index('__init__')]
        class_name = "Feature_Selector"+"."
        for v in self.variables:
            # if the class variable for the argument is not empty, assign its value as the instance variable
            if eval("%s != None" % (class_name+v)) is True:
                exec("self.%s = %s" % (v,class_name+v))
            # if the class variable is empty, assign to instance the value passed as argument during instantiation
            else:
                exec("self.%s = %s" % (v, v))
        
    
        # overriding colsample parameters
        self.best_params['colsample_bytree'] = 1
        self.best_params['colsample_bylevel'] = 1
        
        # determining the model build used for feature selection
        if self.booster == 'gbtree':
            if self.objective in Feature_Selector.classification_objectives:
                self.best_params['objective'] = self.objective
                if 'booster' in list(self.best_params.keys()):
                    del self.best_params['booster']
                self.model = xgb.XGBClassifier(**self.best_params)
            elif self.objective in Feature_Selector.regression_objectives:
                self.best_params['objective'] = self.objective
                if 'booster' in list(self.best_params.keys()):
                    del self.best_params['booster']
                self.model = xgb.XGBRegressor(**self.best_params)
        elif self.booster == 'dart':
            self.best_params['booster'] = self.booster
            self.best_params['objective'] = self.objective
        
        
        # making sure there is a scorer
        if self.scorer not in Feature_Selector.the_higher_the_better and scorer not in Feature_Selector.the_lower_the_better:
            self.scorer = 'Auto'
    
    def fit(self, X, y=None, target=None):
        if y is None:
            if target is None:
                y = pd.DataFrame(X.iloc[:,-1])
                X = pd.DataFrame(X.drop(X.columns[[-1,]], axis=1))
            else:
                y = pd.DataFrame(X[target])
                X = X.drop(self.target, axis =1)
        
        # Encode labels if string when needed
        if self.booster == 'dart' and self.objective in Feature_Selector.classification_objectives or self.scorer == 'f1_score':
            if y.iloc[:,0].dtype == object:
                Target_is_string = True
            else:
                Target_is_string = False
            
            if Target_is_string == True:
                L_E = LabelEncoder()
                L_E = L_E.fit(y.iloc[:,0].astype(str))
                y.iloc[:,0] = L_E.transform(y.iloc[:,0].astype(str))

        # fit model on all training data            
        X_train, X_test, y_train, y_test = train_test_split(X, y,
                                            test_size=self.test_size, random_state=self.random_state, stratify=y)
        
        # Initialize lists of building performance metrics table
        Threshold = []
        ns = []
        Scores = []
        
        # tree boosters: feature selection
        if self.booster == 'gbtree':
            self.test_model = self.model
            self.test_model.fit(X_train, y_train.values.ravel())

            # Fit model using each importance as a threshold
            thresholds = sorted(self.test_model.feature_importances_)
            
            for thresh in thresholds:
                # select features using threshold
                selection = SelectFromModel(self.test_model, threshold=thresh, prefit=True)
                select_X_train = selection.transform(X_train)
                select_X_test = selection.transform(X_test)
                # train model
                if self.objective in Feature_Selector.classification_objectives:
                    estimator = xgb.XGBClassifier
                if self.objective in Feature_Selector.regression_objectives:
                    estimator = xgb.Regressor
                selection_model = estimator(**self.best_params)
                selection_model.fit(select_X_train, y_train.values.ravel(),
                                   eval_set=[(select_X_train,y_train.values.ravel()),
                                             (select_X_test,y_test.values.ravel())], verbose=False)
                
                # eval model
                # using build-in evaluation metrics automatically matched with objective
                if self.scorer == 'Auto':
                    result = selection_model.evals_result()
                    scorer_used = list(result['validation_1'].keys())[0]
                    score = np.mean(result['validation_1'][scorer_used])
                    Scores.append(score)
                    ns.append(len(pd.DataFrame(select_X_train).columns))
                    Threshold.append(thresh)

                # using Sklearn metrics
                else:
                    scoring_function = Feature_Selector.scoring_functions[self.scorer]
                    select_X_test = selection.transform(X_test)
                    y_pred = selection_model.predict(select_X_test)
                    if y_pred.dtype != object:
                        predictions = [round(value) for value in y_pred]
                        y_test_used = [round(value) for value in y_test.iloc[:,0]]
                    else:
                        predictions = y_pred
                        y_test_used = y_test
                    score = scoring_function(y_test_used, predictions)
                    if self.scorer == 'accuracy_score':
                        score = score*100.00
                    Scores.append(score)
                    Threshold.append(thresh)
                    ns.append(select_X_train.shape[1])
        
        # dart boosters: feature selection
        elif self.booster == 'dart':
            dtrain = xgb.DMatrix(data=X_train, label=y_train)
            
            # train the dart booster model
            xg_reg = xgb.train(dtrain=dtrain, params=self.best_params, num_boost_round=10)
            importance = xg_reg.get_fscore()
            importance = sorted(importance.items(), key=operator.itemgetter(1), reverse=True)
            importance = pd.DataFrame(importance)
        
            for value in range(1, len(importance)):
                columns = importance.iloc[:,0][:value]
                dtrain_trim = xgb.DMatrix(data=X_train.loc[:,columns], label=y_train)
                dtest_trim = xgb.DMatrix(data=X_test.loc[:,columns], label=y_test)
                
                # eval model
                # using build-in evaluation metrics automatically matched with objective
                if self.scorer == 'Auto':
                    result = {}
                    selection_model = xgb.train(dtrain=dtrain_trim, params=self.best_params,
                                                evals=[(dtest_trim, 'eval')], evals_result=result,
                                               verbose_eval=False)
                    scorer_used = list(result['eval'].keys())[0]
                    score = np.mean(result['eval'][scorer_used])
                    Scores.append(score)
                    ns.append(len(columns))
                    Threshold.append(importance.iloc[value-1,1])
                
                # using Sklearn metrics
                else:
                    scoring_function = Feature_Selector.scoring_functions[self.scorer]

                    selection_model = xgb.train(dtrain=dtrain_trim, params=self.best_params)
                    y_pred = selection_model.predict(dtest_trim)
                    if y_pred.dtype != object:
                        predictions = [round(value) for value in y_pred]
                        y_test_used = [round(value) for value in y_test.iloc[:,0]]
                    else:
                        predictions = y_pred
                        y_test_used
                    score = scoring_function(y_test_used, predictions)
                    if self.scorer == 'accuracy_score':
                        score = score*100.00
                    Scores.append(score)
                    ns.append(len(columns))
                    Threshold.append(importance.iloc[value-1,1])
        
        # building table of performance scores
        if self.scorer == 'Auto':
            self.scorer = 'Average test ' + scorer_used
        self.performance_scores = pd.DataFrame()
        self.performance_scores['Threshold'] = Threshold
        self.performance_scores['n'] = ns
        self.performance_scores[self.scorer] = Scores

        # Best cut-off of top features: minumum number giving the max possible model performance 
        if self.scorer in Feature_Selector.the_higher_the_better:
            self.best_perf = self.performance_scores[self.performance_scores[self.scorer]\
                                                     == max(self.performance_scores[self.scorer])]
            self.best_threshold = self.best_perf[(self.best_perf['n'] == min(self.best_perf['n']))].drop_duplicates()
        elif self.scorer in Feature_Selector.the_lower_the_better:
            self.best_perf = self.performance_scores[self.performance_scores[self.scorer]\
                                                     == min(self.performance_scores[self.scorer])]
            self.best_threshold = self.best_perf[(self.best_perf['n'] == min(self.best_perf['n']))].drop_duplicates()
        
        # store the best number of features
        self.best_n = self.best_threshold.n.iloc[0]
        
        # saving selected columns for use on .predict()
        if self.booster == 'gbtree':
            self.selected_columns = X.columns[np.argsort(self.test_model.feature_importances_)\
                                          [-(self.best_n):]]
        if self.booster == 'dart':
            self.selected_columns = list(columns[:self.best_n])   
        
        # reverse label encoding
        if self.booster == 'dart' and self.objective in Feature_Selector.classification_objectives:
            if Target_is_string == True:
                y.iloc[:,0] = L_E.inverse_transform(y.iloc[:,0])
        
        return self
        
    def transform(self, X, y=None, target=None):
        if y is None:
            if target is None:
                y = pd.DataFrame(X.iloc[:,-1])
                X = pd.DataFrame(X.drop(X.columns[[-1,]], axis=1))
            else:
                y = pd.DataFrame(X[target])
                X = X.drop(target, axis =1)
        # Drop unwanted features
        X = X[self.selected_columns]
        
        return pd.DataFrame(X), pd.DataFrame(y)
    
    def fit_transform(self, X, y=None, target=None):
        self.fit(X, y, target)
        return self.transform(X, y, target)

In [60]:
Feature_Hyperscaler.booster = 'gbtree'
FS1 = Feature_Hyperscaler(booster='dart')
print(FS1.booster)
Feature_Hyperscaler.booster = 'dart'
FS2 = Feature_Hyperscaler(booster='gblinear')
print(FS2.booster)

gbtree
dart


In [18]:
#             # make predictions for test data and evaluate
#             y_pred = self.test_model.predict(X_test)
#             if y.iloc[:,0].dtype != object:
#                 predictions = [round(value) for value in y_pred]
#             else:
#                 predictions = y_pred
#             accuracy = accuracy_score(y_test, predictions)

ffs = Feature_Selector()
ffs.booster



'gbtree'

## Objectives: guide
https://github.com/dmlc/xgboost/blob/master/doc/parameter.md

In [19]:
class Binary_Undersampler:
    """This transformer is not compatible with the current sklearn Pipeline.
    
    This class is applicable only on data for binary classification.
    This function assumes that 1 is the value of the underrepresented class."""
    def fit(self, X, y):
        return self
    
    def transform(self, X, y):

        # Encoding target variables formatted as string
        if y.iloc[:,0].dtype == object:
            Target_is_string = True
        else:
            Target_is_string = False

        if Target_is_string == True:
            L_E = LabelEncoder()
            L_E = L_E.fit(y.iloc[:,0].astype(str))
            y.iloc[:,0] = L_E.transform(y.iloc[:,0].astype(str))
    
        Positive_count = y.values.sum()
        Positive_indices = np.array(y[y.iloc[:,-1] ==1].index)
        Negative_indices = y[y.iloc[:,-1] == 0].index
        #return X, y
        random_Negative_indices = np.array(np.random.choice(Negative_indices, Positive_count, replace = False))
        Undersample_indices = np.concatenate([Positive_indices, random_Negative_indices])
        
        # Restore original labels
        if Target_is_string == True:
            y.iloc[:,0] = L_E.inverse_transform(y.iloc[:,0])
                
        return X.loc[Undersample_indices,:].reset_index(drop=True), y.loc[Undersample_indices,:].reset_index(drop=True)
        
    def fit_transform(self, X, y):
        self.fit(X, y)
        return self.transform(X, y)

In [20]:
# Class for transforming to dictionary

class Dictionator:
    def __init__(self, orient="records"):
        self.orient = orient
        
    def fit(self, X, y):
        return self
    
    def transform(self, X, y):
        return X.to_dict(self.orient), y.to_dict(self.orient)
        
    def fit_transform(self, X, y):
        self.fit(X, y)
        return self.transform(X, y)
        
        

        
# Class for transforming to DMatrix

In [21]:
class Dictionator:
    classification_objectives = ["reg:logistic"]
    
    def __init__(self, orient="records", objective="reg:linear"):
        self.orient = orient
        self.objective = objective
        
    def fit(self, X, y):
        return self
    
    def transform(self, X, y):
        if self.objective in classification_objectives:
            return X.to_dict(self.orient), y.to_dict(self.orient)
        return X.to_dict(self.orient), y
    
    def fit_transform(self, X, y):
        self.fit(X, y)
        return self.transform(X, y)


In [22]:
# Dummy Variable-creator class
class Dummynatrix:
    classification_objectives = ["reg:logistic"]
        
    def __init__(self, sparse=False, objective="reg:linear"):
        self.sparse = sparse
        self.objective = objective
            
    def fit(self, X, y):
        return self
        
    def transform(self, X, y):
        if self.objective in Dummynatrix.classification_objectives:
            return DictVectorizer(sparse=self.sparse).fit_transform(X), \
                        DictVectorizer(sparse=self.sparse).fit_transform(y)
        return DictVectorizer(sparse=self.sparse).fit_transform(X), y
        
    def fit_transform(self, X, y):
        self.fit(X, y)
        return self.transform(X, y)

In [23]:
# Class for transforming to DMatrix
class DMatrix_Reloader:
    
    def fit(self, data, label):
        return self
    
    def transform(self, data, label):
        return xgb.DMatrix(data= data, label= label )
    
    def fit_transform(self, data, label):
        self.fit(data, label)
        return self.transform(data, label)

In [25]:
# Resampling Class

class HyperSampler:
    """This transformer is not compatible with the current sklearn Pipeline.
    
        Options for resampling methods methods (parameter sampler='') are the following:
            Binary_Undersampler, ClusterCentroids, CondensedNearestNeighbour, EditedNearestNeighbours,
            RepeatedEditedNearestNeighbours, AllKNN, InstanceHardnessThreshold, NearMiss, NeighbourhoodCleaningRule, 
            OneSidedSelection, RandomUnderSampler, TomekLinks, ADASYN, RandomOverSampler, SMOTE, SMOTEENN, 
            SMOTETomek, BalanceCascade, BalancedBaggingClassifier, EasyEnsemble
.
        
        If sampler = 'HyperSampler',
            every resampling method will be applied and tested
            to determine which method best contributes to model performance.
            
            best_params = {dictionary of best parameters for model tuning continuity}
            
            scorer = pass desired Sklearn.metrics scorer as a string
                passing scorer='Auto' entails the use of built-in XGBoost evaluation metrics
            
            The resampling method that contributes to the best score will be applied to the data to be returned.
            
            HyperSampler.resampler = chosen resampling class assigned after instantiation 
                                                                        or after fit() if sampler = 'HyperSampler'
        
        This feature selection transformer works only with any of the three boosters as base learners:
        -gbtree
        -gblinear
        -dart
        
        This class requires that X (features or independent variables) has already been encoded and Imputed. 
        
        Tuning requires the target (y) to also be passed.
        If target = 'insert target name' is passed, X = dataframe disregarding position of target column.
        If the fit_transform method is given one positional argument (X) and target=None, 
            the class assumes that it is a DataFrame with the last
            column as the target variable and the rest as the features."""
    
    # Compatible model evaluation metrics: Sklearn metrics
    scoring_functions = {'accuracy_score':accuracy_score,'f1_score':f1_score, 'hamming_loss':hamming_loss,
        'jaccard_similarity_score':jaccard_similarity_score, 'log_loss':log_loss, 'matthews_corrcoef':matthews_corrcoef,
                 'precision_score':precision_score, 'recall_score':recall_score, 'zero_one_loss':zero_one_loss,
                'explained_variance_score':explained_variance_score, 'mean_absolute_error':mean_absolute_error,
                 'mean_squared_error':mean_squared_error, 'mean_squared_log_error':mean_squared_log_error,
                 'median_absolute_error':median_absolute_error, 'r2_score':r2_score}

    classification_objectives = ['reg:logistic','binary:logistic', 'binary:logitraw',
                                'multi:softmax', 'multi:softprob', 'rank:pairwise' ]
    regression_objectives = ['reg:linear','count:poisson','reg:gamma', 'reg:tweedie']
    
    # metrics with which higher value = higher model performance
    the_higher_the_better = ['accuracy_score','f1_score','jaccard_similarity_score',
                                  'precision_score','recall_score',
                                'explained_variance_score','r2_score','Average test auc',
                                'Average test ndcg','Average test map']
    # metrics with which lower value = higher model performance
    the_lower_the_better = ['hamming_loss', 'log_loss','matthews_corrcoef','zero_one_loss','mean_absolute_error',
                           'mean_squared_error','mean_squared_log_error','median_absolute_error',
                           'Average test error','Average test rmse','Average test mae','Average test log loss',
                           'Average test merror','Average test mlogloss']

    # parameters to inherited from mother class
    sampler=None
    best_params=None
    booster=None
    objective=None
    scorer=None
    
    estimator=None
    voting=None
    n_jobs=None
    return_indices=None
    size_ngh=None
    n_neighbors=None
    n_seeds_S=None
    n_neighbors=None
    kind_sel=None
    max_iter=None
    allow_minority=None
    ratio=None
    cv=None
    version=None
    ver3_samp_ngh=None
    n_neighbors_ver3=None
    threshold_cleaning=None
    replacement=None
    k=None
    n_neighbors=None
    k_neighbors=None
    m=None
    m_neighbors=None
    out_step=None
    kind=None
    svm_estimator=None
    smote=None
    enn=None
    out_step=None
    kind_smote=None
    kind_enn=None
    n_jobs=None
    tomek=None
    n_max_subset=None
    classifier=None
    n_estimators=None
    max_samples=None
    max_features=None
    bootstrap=None
    bootstrap_features=None
    oob_score=None
    warm_start=None
    verbose=None
    n_subsets=None

#     **kwargs
    
    random_state=None
    test_size=None
    
    def __init__(self, scale="HyperSampler", best_params={},
                 booster='gbtree', objective='reg:logistic', scorer='accuracy_score',
                 # parameters for samplers
                 estimator=None,voting='auto',n_jobs=1, return_indices=False,size_ngh=None,n_neighbors=None,n_seeds_S=1,
                 n_neighbors=3,kind_sel='all',max_iter=100,allow_minority=False,ratio='auto',cv=5,version=1,
                 ver3_samp_ngh=None,n_neighbors_ver3=3,threshold_cleaning=0.5,replacement=False,k=None,n_neighbors=5,
                 k_neighbors=5,m=None,m_neighbors=10,out_step=0.5,kind='regular',svm_estimator=None,smote=None,
                 enn=None,out_step=None,kind_smote=None,kind_enn=None,n_jobs=None,tomek=None,n_max_subset=None,
                 classifier=None,n_estimators=10,max_samples=1.0,max_features=1.0,bootstrap=True,bootstrap_features=False,
                 oob_score=False,warm_start=False,verbose=0,n_subsets=10,
                # parameters for train/test split
                 random_state=69, test_size=0.2,
                # values passed for these parameters will never be used unless: class.parameter is made None
                scoring_functions=None, classification_objectives=None,
                 the_higher_the_better=None, the_lower_the_better=None):
        
        # assigning parameters as instance variables
        varses = list(vars(HyperSampler).keys())
        self.variables = varses[len(varses) - varses[::-1].index('__doc__') : varses.index('__init__')]
        class_name = "HyperSampler"+"."
        for v in self.variables:
            # if the class variable for the argument is not empty, assign its value as the instance variable
            if eval("%s != None" % (class_name+v)) is True:
                exec("self.%s = %s" % (v,class_name+v))
            # if the class variable is empty, assign to instance the value passed as argument during instantiation
            else:
                exec("self.%s = %s" % (v, v))
        
        # determining the model build used for feature selection
        if self.booster == 'gbtree':
            if self.objective in HyperSampler.classification_objectives:
                self.best_params['objective'] = self.objective
                if 'booster' in list(self.best_params.keys()):
                    del self.best_params['booster']
                self.model = xgb.XGBClassifier(**self.best_params)

            elif self.objective in HyperSampler.regression_objectives:
                self.best_params['objective'] = self.objective
                if 'booster' in list(self.best_params.keys()):
                    del self.best_params['booster']
                self.model = xgb.XGBRegressor(**self.best_params)
        elif self.booster == 'dart' or self.booster == 'gblinear':
            self.best_params['booster'] = self.booster
            self.best_params['objective'] = self.objective
        
        self.sampler_params={'StandardScaler':[StandardScaler,{'copy':self.copy, 
                                              'with_mean':self.with_mean, 'with_std':self.with_std}],
                          'MinMaxScaler':[MinMaxScaler,{'feature_range':self.feature_range, 'copy':self.copy}],
                          'Normalizer':[Normalizer,{'norm':self.norm, 'copy':self.copy}],
                            'RobustScaler':[RobustScaler,{'with_centering':self.with_centering,
                                            'with_scaling':self.with_scaling,
                                             'quantile_range':self.quantile_range, 'copy':self.copy}],
                           'MaxAbsScaler':[MaxAbsScaler,{'copy':self.copy}],
                           'QuantileTransformer':[QuantileTransformer,{'n_quantiles':self.n_quantiles, 
                                                  'output_distribution':self.output_distribution,
                                                  'ignore_implicit_zeros':self.ignore_implicit_zeros,
                                                  'subsample':self.subsample,'random_state':self.random_state}]}
        
        # Assigning the resampler if sampler (sampling method) is not "Hyperscale"
        if self.sampler is not 'HyperSampler':
            self.resampler = self.sampler_params[self.sampler][0](**self.sampler_params[self.sampler][1])
    
    def fit(self, X, y=None, target=None):
        if y is None:
            if self.target is None:
                y = pd.DataFrame(X.iloc[:,-1])
                X = pd.DataFrame(X.drop(X.columns[[-1,]], axis=1))
            else:
                y = pd.DataFrame(X[self.target])
                X = X.drop(target, axis =1)
        
        if self.sampler == 'HyperSampler':
            # Encode labels if string when needed
            if self.booster == 'dart' and self.objective in HyperSampler.classification_objectives\
                                                                    or self.scorer == 'f1_score':
                if y.iloc[:,0].dtype == object:
                    Target_is_string = True
                else:
                    Target_is_string = False

                if Target_is_string == True:
                    L_E = LabelEncoder()
                    L_E = L_E.fit(y.iloc[:,0].astype(str))
                    y.iloc[:,0] = L_E.transform(y.iloc[:,0].astype(str))

            # fit model on all training data            
            X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                test_size=self.test_size, random_state=self.random_state, stratify=y)
            
            samplers = ['None','Binary_Undersampler', 'ClusterCentroids', 'CondensedNearestNeighbour', 'EditedNearestNeighbours',
            'RepeatedEditedNearestNeighbours', 'AllKNN', 'InstanceHardnessThreshold', 'NearMiss', 'NeighbourhoodCleaningRule', 
            'OneSidedSelection', 'RandomUnderSampler', 'TomekLinks', 'ADASYN', 'RandomOverSampler', 'SMOTE', 'SMOTEENN', 
            'SMOTETomek', 'BalanceCascade', 'BalancedBaggingClassifier', 'EasyEnsemble']
            
            Scores = []
            for sampler in samplers:
                if sampler is 'None':
                    X_train_resampled, y_train_resampled = X_train, y_train
                    X_test_resampled, y_test_resampled = X_test, y_test
                else:
                    sampler = self.sampler_params[sampler][0](**self.sampler_params[sampler][1])
                    X_train_resampled, y_train_resampled = sampler.fit_sample(X_train, y_train)
                    X_test_resampled, y_test_resampled = sampler.fit_sample(X_test, y_test)
                
                if self.booster is 'gbtree':
                    if self.objective in HyperSampler.classification_objectives:
                        estimator = xgb.XGBClassifier
                    if self.objective in HyperSampler.regression_objectives:
                        estimator = xgb.Regressor
                    selection_model = estimator(**self.best_params)
                    selection_model.fit(X_train_scaled, y_train.values.ravel(),
                                       eval_set=[(X_train_resampled,y_train_resampled.values.ravel()),
                                                 (X_test_resampled,y_test_resampled.values.ravel())], verbose=False)
                    # eval model
                    # using build-in evaluation metrics automatically matched with objective
                    if self.scorer == 'Auto':
                        result = selection_model.evals_result()
                        scorer_used = list(result['validation_1'].keys())[0]
                        score = np.mean(result['validation_1'][scorer_used])
                        Scores.append(score)

                    # using Sklearn metrics
                    else:
                        scoring_function = HyperSampler.scoring_functions[self.scorer]
                        y_pred = selection_model.predict(X_test_scaled)
                        if y_pred.dtype != object:
                            predictions = [round(value) for value in y_pred]
                            y_test_used = [round(value) for value in y_test.iloc[:,0]]
                        else:
                            predictions = y_pred
                            y_test_used = y_test
                        score = scoring_function(y_test_used, predictions)
                        if self.scorer == 'accuracy_score':
                            score = score*100.00
                        Scores.append(score)
                        
                elif self.booster == 'dart' or self.booster == 'gblinear':
                    dtrain_resampled = xgb.DMatrix(data=X_train_resampled, label=y_train_resampled)
                    dtest_resampled = xgb.DMatrix(data=X_test_resampled, label=y_test_resampled)

                    # eval model
                    # using build-in evaluation metrics automatically matched with objective
                    if self.scorer == 'Auto':
                        result = {}
                        selection_model = xgb.train(dtrain=dtrain_resampled, params=self.best_params,
                                                    evals=[(dtest_resampled, 'eval')], evals_result=result,
                                                   verbose_eval=False)
                        scorer_used = list(result['eval'].keys())[0]
                        score = np.mean(result['eval'][scorer_used])
                        Scores.append(score)

                    # using Sklearn metrics
                    else:
                        scoring_function = HyperSampler.scoring_functions[self.scorer]

                        selection_model = xgb.train(dtrain=dtrain_resampled, params=self.best_params)
                        y_pred = selection_model.predict(dtest_resampled)
                        if y_pred.dtype != object:
                            predictions = [round(value) for value in y_pred]
                            y_test_used = [round(value) for value in y_test.iloc[:,0]]
                        else:
                            predictions = y_pred
                            y_test_used = y_test
                        score = scoring_function(y_test_used, predictions)
                        if self.scorer == 'accuracy_score':
                            score = score*100.00
                        Scores.append(score)

            # building table of performance scores
            if self.scorer == 'Auto':
                self.scorer = 'Average test ' + scorer_used
            self.performance_scores = pd.DataFrame()
            self.performance_scores['Samplers'] = samplers
            self.performance_scores[self.scorer] = Scores

            # Best scaler: returns max possible model performance 
            if self.scorer in HyperSampler.the_higher_the_better:
                self.best_perf = self.performance_scores[self.performance_scores[self.scorer]\
                                                         == max(self.performance_scores[self.scorer])]
            elif self.scorer in HyperSampler.the_lower_the_better:
                self.best_perf = self.performance_scores[self.performance_scores[self.scorer]\
                                                         == min(self.performance_scores[self.scorer])]
                
            # assigning the best-performing scaler as scaler
            self.best_sampler = self.best_perf.iloc[0,0]
            if self.best_sampler is not 'None':
                self.resampler = self.sampler_params[self.best_sampler][0](**self.sampler_params[self.best_sampler][1])
            else:
                self.resampler = None
            
            # reverse label encoding
            if self.booster == 'dart' and self.objective in HyperSampler.classification_objectives:
                if Target_is_string == True:
                    y.iloc[:,0] = L_E.inverse_transform(y.iloc[:,0])
            
            return self
            
        else:
            return self
    
    def sample(self, X, y=None, target=None):
        if y is None:
            if target is None:
                y = pd.DataFrame(X.iloc[:,-1])
                X = pd.DataFrame(X.drop(X.columns[[-1,]], axis=1))
            else:
                y = pd.DataFrame(X[target])
                X = X.drop(target, axis =1)
        if self.best_sampler is not 'None':
            X_resampled, y_resampled = self.resampler.fit_sample(X, y)
            return pd.DataFrame(X_resampled, columns=X.columns), pd.DataFrame(y_resampled, columns=y.columns)
        else:
            return pd.DataFrame(X), pd.DataFrame(y)
        
    def fit_sample(self, X, y=None, target=None):
        self.fit(X,y, target=target)
        return self.sample(X, y, target=target)

In [69]:
print(Imputation_Nation.__doc__)

This class handles the imputation of missing values
        and the encoding and creation of dummy variables for categorical variables.

        If the target variable is passed as y, it will also be imputed if impute_y= True.
        Otherwise, y will pass through as untouched.


In [26]:
# Mushroom = pd.read_csv('mushrooms.csv')#.dropna()
# df = Mushroom
# X = Mushroom.drop('class',axis=1)
# y = pd.DataFrame(Mushroom['class'])
# print(type(X))
#print(y)

df = pd.read_csv('creditcard.csv')
df = df.drop('Time', axis=1)
df.reset_index(inplace=True)
target_name = 'Class'
X = df.drop(target_name, axis =1)
y = pd.DataFrame(df[target_name])

In [27]:
X.head(10)

,index,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62
1,1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69
2,2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66
3,3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50
4,4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99
5,5,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,...,0.084968,-0.208254,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,3.67
6,6,1.229658,0.141004,0.045371,1.202613,0.191881,0.272708,-0.005159,0.081213,0.464960,...,-0.219633,-0.167716,-0.270710,-0.154104,-0.780055,0.750137,-0.257237,0.034507,0.005168,4.99
7,7,-0.644269,1.417964,1.074380,-0.492199,0.948934,0.428118,1.120631,-3.807864,0.615375,...,-0.156742,1.943465,-1.015455,0.057504,-0.649709,-0.415267,-0.051634,-1.206921,-1.085339,40.80
8,8,-0.894286,0.286157,-0.113192,-0.271526,2.669599,3.721818,0.370145,0.851084,-0.392048,...,0.052736,-0.073425,-0.268092,-0.204233,1.011592,0.373205,-0.384157,0.011747,0.142404,93.20
9,9,-0.338262,1.119593,1.044367,-0.222187,0.499361,-0.246761,0.651583,0.069539,-0.736727,...,0.203711,-0.246914,-0.633753,-0.120794,-0.385050,-0.069733,0.094199,0.246219,0.083076,3.68


In [28]:
list(y.iloc[:,0].unique())

[0, 1]

In [29]:
# y_nulled = Nullifier().df_null_dictionary(Mushroom, {'class':['e']})
# y_nulled

In [30]:
X_nonull, y_nonull = MCAR_Dropper().fit_transform(X, y)
y_nonull.iloc[:,0].unique()
type(y_nonull)

pandas.core.frame.DataFrame

In [31]:
X_imputed = Imputation_Nation(objective='reg:logistic').fit_transform(X_nonull)
y_imputed = Imputation_Nation().fit_transform(y_nonull)
#X_imputed

In [32]:
X_imputed, y_imputed = Imputation_Nation(objective="reg:logistic", impute_y=True).fit_transform(X, y)
y_imputed.head(10)

,Class
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0


In [33]:
from collections import Counter
X_undersampled, y_undersampled = Binary_Undersampler().fit_transform(X_imputed, y_imputed)
print(len(X_undersampled))
print(len(y_undersampled))
print(Counter(y_undersampled.iloc[:,0]))

984
984
Counter({1: 492, 0: 492})


In [34]:
Counter(y_undersampled.iloc[:,0])

Counter({0: 492, 1: 492})

In [35]:
scales = ['None','StandardScaler','MinMaxScaler','Normalizer',
          'RobustScaler','MaxAbsScaler','QuantileTransformer']
FSS = Feature_Hyperscaler(booster='dart',objective='binary:logistic', scorer='Auto')
scale = 'MaxAbsScaler'

FSS.fit(X_undersampled, y_undersampled)
X_scaled, y_scaled = FSS.transform(X_undersampled, y_undersampled)
y_scaled.iloc[:,0].unique()

pd.DataFrame(X_scaled).head(10)
FSS.performance_scores

,Scalers,Average test error
0,None,0.043452
1,StandardScaler,0.070863
2,MinMaxScaler,0.104772
3,Normalizer,0.060508
4,RobustScaler,0.064061
5,MaxAbsScaler,0.056244
6,QuantileTransformer,0.066345


In [36]:
FSS.best_perf

,Scalers,Average test error
0,None,0.043452


In [37]:
FSS.scaler

In [38]:
y_scaled.head(5)

,Class
0,1
1,1
2,1
3,1
4,1


In [64]:
parms = {'booster':'dart','objective':'reg:linear'}
dtrain = xgb.DMatrix(data=X_scaled, label=y_scaled)
linear = xgb.train(dtrain=dtrain, params=parms)
linear.get_fscore()

{'Amount': 11,
 'V1': 20,
 'V10': 12,
 'V11': 6,
 'V12': 19,
 'V13': 2,
 'V14': 29,
 'V15': 3,
 'V16': 5,
 'V17': 12,
 'V18': 3,
 'V19': 11,
 'V2': 11,
 'V20': 6,
 'V21': 8,
 'V22': 8,
 'V23': 6,
 'V24': 4,
 'V25': 4,
 'V26': 3,
 'V27': 3,
 'V28': 3,
 'V3': 15,
 'V4': 40,
 'V5': 7,
 'V6': 3,
 'V7': 13,
 'V8': 8,
 'V9': 4,
 'index': 35}

In [40]:
#y_scaled.iloc[:,0] = pd.get_dummies(y_scaled.iloc[:,0], drop_first=True, columns=[y_scaled.columns[0]])kwargs = {"colsample_bytree":0.8,
       #"gamma":0.1, "learning_rate":0.2, "colsample_bylevel":0.9}
#model = Feature_Selector(objective='count:poisson', best_params=kwargs).fit(X_scaled, y_scaled).test_model
#print(model)
#print(xgb.XGBClassifier())

feat_selector = Feature_Selector(booster='dart',objective='binary:logistic', scorer='Auto')
feat_selector = feat_selector.fit(X_scaled, y_scaled)


X_processed, y_processed = feat_selector.transform(X_scaled, y_scaled)
#print(feat_selector.test_model.feature_importances_)
# print(X_scaled.columns[np.argsort(feat_selector.test_model.feature_importances_)[-(feat_selector.best_threshold.n.iloc[0]):]])
X_processed.head(10)

,V14,V4,V8,V19,V12,V16,V20,V23,V17,V10,Amount
0,-4.289254,3.997906,1.391657,0.416956,-2.899907,-1.140747,0.126911,-0.465211,-2.830056,-2.772272,0.00
1,-1.692029,2.288644,-0.067794,0.283345,-0.503141,0.666780,2.102339,1.375966,0.599717,-0.838587,529.00
2,-1.470102,2.330243,-0.399147,-1.334441,-6.560124,-2.282194,-0.430022,0.172726,-4.781831,-1.525412,239.93
3,-6.771097,2.679787,-0.248778,0.308334,-10.912819,-7.358083,-0.171608,-0.436207,-12.598419,-4.801637,59.00
4,-6.079337,4.732795,-0.496358,-2.721853,-4.609628,2.581851,0.009061,-0.656805,6.739384,-2.447469,1.00
5,-10.691196,6.675732,0.154612,-1.934666,-9.854485,-2.041974,0.488378,-0.539528,-1.129056,-6.187891,1.00
6,-10.733854,6.348557,0.303253,-1.327357,-8.948179,-1.638960,0.587743,-0.669605,-1.746350,-6.045468,1.00
7,-9.177166,6.078266,0.133080,-2.370599,-8.873748,-0.871688,0.269773,-0.551572,1.313014,-5.134454,1.00
8,-9.252307,6.047445,0.055586,-1.808012,-7.520117,-0.502362,0.388307,-0.583813,0.784427,-4.959493,1.00
9,-6.210258,4.007683,1.063728,2.250123,-7.148243,-3.599540,0.504646,0.601045,-4.830324,-4.624985,1.00


In [41]:
feat_selector.performance_scores

,Threshold,n,Average test error
0,20,1,0.084264
1,18,2,0.059899
2,11,3,0.058884
3,10,4,0.060914
4,9,5,0.064467
5,8,6,0.053807
6,7,7,0.067005
7,7,8,0.062436
8,7,9,0.069543
9,6,10,0.059899


In [42]:
feat_selector.best_threshold

,Threshold,n,Average test error
10,6,11,0.051776


In [43]:
y_processed.head(5)

,Class
0,1
1,1
2,1
3,1
4,1


In [44]:
X_done= PCA().fit_transform(X_processed)
Counter([type(value) for value in pd.DataFrame(X_done).columns])

Counter({numpy.int64: 11})

In [45]:
pd.DataFrame(X_done)

,0,1,2,3,4,5,6,7,8,9,10
0,-98.856112,0.278069,-1.088056,-1.487752,0.771298,1.008360,-0.686408,0.491947,0.385315,0.093287,-0.522633
1,430.136945,-6.091276,0.415409,-1.705227,1.505790,0.512664,-1.236220,-1.386649,-1.130202,1.139248,-0.233847
2,141.074143,0.689728,0.739069,1.181097,-0.449276,1.799765,3.680595,0.337945,-1.897309,-0.199442,0.201884
3,-39.843731,12.333006,1.114797,3.029670,-3.653070,0.582510,3.440398,0.487783,-1.248633,0.690207,0.589026
4,-97.871374,-5.085720,1.098542,-10.053107,3.371072,-0.215647,2.442506,0.213195,-0.049082,-0.450416,-0.102209
5,-97.859471,6.989140,1.433912,-8.721733,1.717686,-0.275183,2.668328,0.223085,-0.537166,0.061027,0.281210
6,-97.858743,6.739877,1.141540,-8.139220,1.388232,-0.450660,1.782010,0.468718,-0.367861,0.356872,-0.287447
7,-97.862924,3.622315,1.216305,-9.168162,2.304519,-0.233600,3.181717,0.072721,-0.495574,-0.209742,0.336258
8,-97.862472,3.193638,1.153358,-8.662065,2.059091,-0.212865,1.857561,0.375717,-0.273749,0.011583,-0.082094
9,-97.852374,5.612611,-0.155147,-1.406663,-0.193498,0.297211,0.908549,-2.039980,0.497813,0.335155,-0.924666


# COPY
# XGBoost Model Trainer (custom decorator function)

def XGBoost_Trainerrrr(df, target=None, model= 'other', resampling=None,
                    booster="gbtree", objective="reg:linear",
                    test_size=0.2,
                    random_state=69,
                    normalization=True,
                    pca=False,
                    hyperparameter_test= "none",
                    cv_metric="rmse",
                    missing_values="NaN",
                    imputation_strategy="mean",
                    n_iter= 25,
                    hypertune_folds= 5,
                    cv_folds= 5,
                    scoring="neg_mean_squared_error",
                    grid_params = "preset",
                    random_params = "preset"):
    """This function requires the xgboost package to be imported as xgb."""
    
    if model == "Tree Regressor":
        bj = ['gbtree','reg:linear']
    elif model == "Linear Regressor":
        bj = ['gblinear','reg:linear']
    elif model == "Tree Classifier":
        bj = ['gbtree', 'reg:logistic']
    elif model == "Linear Classifier":
        bj = ['gblinear', 'reg:logistic']
    elif mode == 'other':
        bj = [booster, objective]
    
    grid_parameters = {}
    random_parameters = {}
    
    if grid_params == "preset":
        grid_params = grid_parameters
    if random_params == "preset":
        random_params == random_parameters
    
    regression_objectives=[]
    classification_objectives = ["reg:logistic"]

    # Data Preprocessing Pipeline Steps
    preprocessing_steps = [
            ("Imputer", Imputer(missing_values=missing_values, strategy= imputation_strategy, axis=0))
                            ]
    
    # Scaling
    if normalization == True:
        preprocessing_steps = preprocessing_steps + [
            ("Scaler", StandardScaler())
                            ]
    
    # Dummy Variable Creation
    class Dictionator:
        def __init__(self, orient="records"):
            self.orient = orient
        
        def fit(self, X, y):
            return self
    
        def transform(self, X, y):
            return X.to_dict(self.orient), y.to_dict(self.orient)
        
        def fit_transform(self, X, y):
            self.fit(X, y)
            return self.transform(X, y)
    
    
    preprocessing_steps = preprocessing_steps + [ 
        ("Encoder", FeatureUnion(
            [("Label",LabelEncoder()),
             ("Hot", OneHotEncoder())]))
                            ]  
    
    #### define a CLASS for undersampling
    class Binary_Undersampler:
        """This class is applicable only on data for binary classification.
        This class assumes that 1 is the value of the underrepresented class."""
        def fit(self, X, y):
            return self
    
        def transform(self, X, y):
            y = y.values.ravel()
            self.Positive_count = y.values.sum()
            self.Positive_indices = np.array(y[y.iloc[:,-1] ==1].index)
            self.Negative_indices = y[y.iloc[:,-1] == 0].index
            self.random_Negative_indices = np.array(np.random.choice(self.Negative_indices, self.Positive_count,
                                                                     replace = False))
            self.Undersample_indices = np.concatenate([self.Positive_indices, self.random_Negative_indices])
            return X.loc[self.Undersample_indices,:], y.loc[self.Undersample_indices,:]
        
        def fit_transform(self, X, y):
            self.fit(X, y)
            return self.transform(X, y)

    
    # Resampling Method
    if objective in classification_objectives or model in ["Tree Classifier","Linear Classifier"]:
        if resampling == "Undersampling":
            preprocessing_steps = preprocessing_steps + [("The Undersampler", Binary_Undersampler())]
        elif resampling == "SMOTE":
            preprocessing_steps = preprocessing_steps + [("The Oversampler", SMOTE())]
            
    # Principal Component Analysis (PCA)
    if pca == True:
        preprocessing_steps = preprocessing_steps + [("PCA", PCA())]
    
    # Separating features and target
    X, y = df[df.columns.tolist()[:,-1]], df[df.columns.tolist()[-1]]
    
    # Split between test and training sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)

    
    # Completing the pipeline with the appropriat XGBoost Model
    # correct version
    if bj[0] == "gblinear":
        DM_train = xgb.DMAtrix(data=X_train, label=y_train)
        if hyperparameter_test == "Grid Search":
            pipeline = Pipeline[
                preprocessing_steps,
                ("Tuner", GridSearchCV(estimator=xgb.train(dtrain=DM_train, booster=bj[0], objective=bj[1]),
                                      param_grid= grid_params,
                                      scoring= scoring,
                                      cv= hypertune_folds,
                                      verbose=1))
                        ]
        elif hyperparameter_test == "Random Search":
            pipeline = Pipeline[
                preprocessing_steps,
                ("Tuner", RandomizedSearchCV(estimator=xgb.train(dtrain=DM_train, booster=bj[0], objective=bj[1]),
                                                                param_distributions=random_parameters,
                                                                scoring= scoring,
                                                                cv = hypertune_folds,
                                                                n_iter= n_iter))
                        ]
        elif hyperparameter_test == "none":
            pipeline = Pipeline[
                preprocessing_steps,
                ("Model", xgb.train(dtrain=DM_train, booster=bj[0], objective=bj[1]))
                        ]
    
    elif bj[0] == "gbtree":
        if bj[1] in regression_objectives:
            if hyperparameter_test == "Grid Search":
                pipeline = Pipeline[
                    preprocessing_steps,
                    ("Tuner", GridSearchCV(estimator=xgb.XGBRegressor(booster=bj[0], objective=bj[1]),
                                          param_grid= grid_params,
                                          scoring= scoring,
                                          cv= hypertune_folds,
                                          verbose=1))
                        ]
            elif hyperparameter_test == "Random Search":
                pipeline = Pipeline[
                    preprocessing_steps,
                    ("Tuner", RandomizedSearchCV(estimator=xgb.XGBRegressor(booster=bj[0], objective=bj[1]),
                                                                           param_distributions=random_parameters,
                                                                           scoring= scoring,
                                                                           cv = hypertune_folds,
                                                                           n_iter= n_iter))
                        ]
            elif hyperparameter_test == "none":
                pipeline = Pipeline[
                    preprocessing_steps,
                    ("Model", xgb.XGBRegressor(booster=bj[0], objective=bj[1]))
                        ]

        elif bj[1] in classification_objectives:
            if hyperparameter_test == "Grid Search":
                pipeline = Pipeline[
                    preprocessing_steps,
                    ("Tuner", GridSearchCV(estimator=xgb.XGBClassifier(booster=bj[0], objective=bj[1]),
                                          param_grid= grid_params,
                                          scoring= scoring,
                                          cv= hypertune_folds,
                                          verbose=1))
                        ]
            elif hyperparameter_test == "Random Search":
                pipeline = Pipeline[
                    preprocessing_steps,
                    ("Tuner", RandomizedSearchCV(estimator=xgb.XGBClassifier(booster=bj[0], objective=bj[1]),
                                                                           param_distributions=random_parameters,
                                                                           scoring= scoring,
                                                                           cv = hypertune_folds,
                                                                           n_iter= n_iter))
                        ]
            elif hyperparameter_test == "none":
                pipeline = Pipeline[
                    preprocessing_steps,
                    ("Model", xgb.XGBClassifier(booster=bj[0], objective=bj[1]))
                        ]             
    
    
    
    # fitting the pipeline 
    pipeline.fit(X_train.to_dict("records"), y_train)
    
    if hyperparameter_test == "Grid Search" or hyperparameter_test == "Random Search":
        if bj[0] == "gblinear":
            xgb_model = xgb.XGBRegressor(booster="gbtree", objective="reg:linear", **pipeline.best_params_)
            xgb_model.fit(X_train, y_train)
        elif model == "Linear Regressor":
            xgb_model == xgb.train(booster="gblinear", objective='reg:linear', **pipeline.best_params_)
            xgb_model.fit(X_train, y_train)
        elif model == "Tree Classifier":
            xgb_model == XGBClassifier(booster="gbtree")

            
        return pd.concat([pd.DataFrame(Imputer(missing_values= self.missing_values,\
                                                           strategy=self.num_imput_strat,\
                                   axis=self.imputation_axis).fit_transform(X.drop(df.columns[[-1,]], axis=1)\
                                                    .drop(X.index[pd.isnull(X.iloc[:,-1]).any(1).nonzero()[0]]))), \
                                pd.DataFrame(X.iloc[:,-1]).dropna()], axis=1)
